# ORM의 특징

## 1. Lazy Loading 지연로딩 : 정말 필요한 시점에 sql을 호출한다.
### 1-1. 실제 데이터가 필요한 시점에 SQL을 호출한다.
  - `User.objects.all()` 시점에서는 실제로 사용하지 않지 때문에, SQL을 호출하지 않지만 list로 변환할때는 실제 데이터가 필요하므로 SQL을 호출한다.

In [12]:
users = User.objects.all()

In [13]:
users = User.objects.all()
user_list = list(users)

(0.014) SELECT "users"."id", "users"."email", "users"."name" FROM "users"; args=(); alias=default


### 1-2. 직접 사용하지 않는 쿼리셋은 호출하지 않는다.
  - 쿼리셋을 3개를 만들었지만 사용한(seller) 쿼리셋만 SQL을 호출함

In [16]:
users = User.objects.all()
products = Product.objects.all()
sellers = Seller.objects.all()

seller_list = list(sellers)

(0.013) SELECT "sellers"."id", "sellers"."email", "sellers"."name", "sellers"."phone_number" FROM "sellers"; args=(); alias=default


### 1.3 불필요한 SQL 호출이 생길 수 있다.
  - user 1명만 조회 후, 모든 user를 조회 해야된다고 하면, 1명만 조회하는 SQL, 모두 조회하는 SQL 총 2번 SQL을 호출한다.
  - 이는 순서만 바꿔도 모두 조회하는 SQL을 한번만 호출하면 된다.
  - 모든 user를 가져오라는 sql이 호출되서 모든 user 데이터가 캐싱되어 있기 때문이다.
  - ORM입장에서는 현 시점에서 가장 효율적인 SQL을 호출하려고 하기 때문이다. ORM은 전체 흐름을 알 수 없음!

In [23]:
users = User.objects.all()
first_user = users[0]
all_user = list(users)

(0.013) SELECT "users"."id", "users"."email", "users"."name" FROM "users" LIMIT 1; args=(); alias=default
(0.003) SELECT "users"."id", "users"."email", "users"."name" FROM "users"; args=(); alias=default


In [25]:
users = User.objects.all()
all_user = list(users) # 모든 user 캐싱
first_user = users[0]

(0.051) SELECT "users"."id", "users"."email", "users"."name" FROM "users"; args=(); alias=default


### 1.4 N+1 problem가 발생할 수 있다.
  - 모든 product의 정보를 가져오라고 하고, for문을 통해서 연관된 판매자 정보까지 출력하려하면 
  - for문을 도는 시점에 해당상품에 연관된 판매자 한사람만의 정보를 필요하기 때문에 상품수만큼 판매자를 알아내는 SQL문이 호출된다.
  - 그래서 총 상품전체조회하는 SQL + 판매자정보를 상품개수만큼 호출하는 SQL 이 발생하기 때문에, N+1문제라고 부른다.
  - 이를 해결하기 위해서는 Enger Loading(즉시호출) 사용한다.

In [37]:
products = Product.objects.all()
product_sellers = [{'name' : product.name, 'seller' : product.related_seller.name} for product in products]

(0.015) SELECT "products"."id", "products"."name", "products"."price", "products"."related_seller_id" FROM "products"; args=(); alias=default
(0.005) SELECT "sellers"."id", "sellers"."email", "sellers"."name", "sellers"."phone_number" FROM "sellers" WHERE "sellers"."id" = 1 LIMIT 21; args=(1,); alias=default
(0.009) SELECT "sellers"."id", "sellers"."email", "sellers"."name", "sellers"."phone_number" FROM "sellers" WHERE "sellers"."id" = 2 LIMIT 21; args=(2,); alias=default
(0.022) SELECT "sellers"."id", "sellers"."email", "sellers"."name", "sellers"."phone_number" FROM "sellers" WHERE "sellers"."id" = 3 LIMIT 21; args=(3,); alias=default
(0.003) SELECT "sellers"."id", "sellers"."email", "sellers"."name", "sellers"."phone_number" FROM "sellers" WHERE "sellers"."id" = 4 LIMIT 21; args=(4,); alias=default
(0.003) SELECT "sellers"."id", "sellers"."email", "sellers"."name", "sellers"."phone_number" FROM "sellers" WHERE "sellers"."id" = 5 LIMIT 21; args=(5,); alias=default
(0.003) SELECT "se

- select_related와 prefetch_related를 사용해서 N+1 problem 해결할 수 있다.

In [152]:
products = Product.objects.select_related('related_seller')
product_sellers = [{'name' : product.name, 'seller' : product.related_seller.name} for product in products]

(0.004) SELECT "products"."id", "products"."name", "products"."price", "products"."related_seller_id", "sellers"."id", "sellers"."email", "sellers"."name", "sellers"."phone_number" FROM "products" INNER JOIN "sellers" ON ("products"."related_seller_id" = "sellers"."id"); args=(); alias=default


In [150]:
queryset = Seller.objects.filter(id__gte=5)
products = Product.objects.prefetch_related('related_seller')

product_sellers = [{'name':product.name, 'seller':product.related_seller.name} for product in products]

(0.030) SELECT "products"."id", "products"."name", "products"."price", "products"."related_seller_id" FROM "products"; args=(); alias=default
(0.006) SELECT "sellers"."id", "sellers"."email", "sellers"."name", "sellers"."phone_number" FROM "sellers" WHERE "sellers"."id" IN (1, 2, 3, 4, 5, 6, 7, 8, 9, 10); args=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10); alias=default


## 2.1 Queryset
Queryset class에는 다음과 같은 값들이 포함되어 있다
- query: 메인 쿼리
- _prefetch_related_lookups : 추가 쿼리 참조값
- _result_cache : 현재 캐쉬된 값들

In [185]:
products = Product.objects.prefetch_related('related_seller')
print(f'\nmain query : {products.query}') # 메인 쿼리
print(f'\nresult cache : {products._result_cache}') # 캐시된 값들
print(f'\nadditional queryset : {products._prefetch_related_lookups}') # 추가 쿼리셋
len(products)
print(f'\nresult cache after call query : {products._result_cache}') # 캐시된 값들 (쿼리 호출 후)


(0.010) SELECT "products"."id", "products"."name", "products"."price", "products"."related_seller_id" FROM "products"; args=(); alias=default
(0.016) SELECT "sellers"."id", "sellers"."email", "sellers"."name", "sellers"."phone_number" FROM "sellers" WHERE "sellers"."id" IN (1, 2, 3, 4, 5, 6, 7, 8, 9, 10); args=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10); alias=default



main query : SELECT "products"."id", "products"."name", "products"."price", "products"."related_seller_id" FROM "products"

result cache : None

additional queryset : ('related_seller',)

result cache after call query : [<Product: Product object (1)>, <Product: Product object (2)>, <Product: Product object (3)>, <Product: Product object (4)>, <Product: Product object (5)>, <Product: Product object (6)>, <Product: Product object (7)>, <Product: Product object (8)>, <Product: Product object (9)>, <Product: Product object (10)>]
